In [2]:
import sys
sys.path.insert(0, '..')
# Useful starting lines
%matplotlib inline
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2

In [3]:
import Helpers.helpers as helper
import Helpers.cleaner as cleaner
import numpy as np
import Helpers.implementations as imp

In [4]:
part = [0,1]
data = []
for p in part:
    data.append(helper.load_clean_data(f'../../data/clean/x_train_leo_{p}.npy', f'../../data/clean/y_train_leo_{p}.npy'))

In [5]:
def toDeg(x,degree):
    res = x
    for i in range(2,degree +1):
        power = np.power(x,i)
        res = np.c_[res,power]
    return res

def addConstant(x):
    return np.c_[np.ones((x.shape[0],1)),x]


def add_cos(x, dim):
    """Only do it after adding a constant."""
    select = x[:,1:dim+1]
    cos = np.cos(select)
    return np.concatenate([x, cos], axis=1)


def add_sin(x, dim):
    """Only do it after adding a constant."""
    select = x[:,1:dim+1]
    sin = np.sin(select)
    return np.concatenate([x, sin], axis=1)

def add_effect(x, dim, fun):
    """Only do it after adding a constant."""
    select = x[:,1:dim+1]
    ret = fun(select)
    return np.concatenate([x, ret], axis=1)

In [10]:
x = [data[p][0] for p in part]
y = [data[p][1] for p in part]

xpow = [helper.addConstant(helper.toDeg(x[p], 2)) for p in part]

In [29]:
def train_model_ridge_cross(x, y, lambdas=[-0.00001], ratio=.95, seed=1):
    accuracies = []
    ws = []
    rmse_trs = []
    rmse_tes = []
    
    x_tr,y_tr,x_te,y_te = imp.split_data(x,y,ratio,seed)
    w_all, rmse_tr, rmse_te = imp.full_cross_validation(x_tr,y_tr, lambdas=lambdas)
    
    for w in w_all:
        y_pred = helper.predict_labels(w,x_te)
        res = np.array([(1 if(y_pred[i] == y_te[i]) else 0) for i in range(y_te.shape[0])])
        accuracies.append(res.sum()/len(res))
        
        
    i = np.argmax(accuracies)
    return w_all[i], accuracies[i], rmse_tr[i], rmse_te[i]

In [30]:
def train_all_models(xs, ys, lambdas=[0.001, .01, -0.01], parts=[0,1,2,3]):
#     dim = x.shape[1]
    xpow = [helper.addConstant(helper.toDeg(xs[p], 2)) for p in parts]
    
    ws = []
    accuracies = []
    rmse_trs = []
    rmse_tes = []
    for p in parts:
        dim = xpow[p].shape[1]
        xpow_ = xpow[p]
#         xpow_ = helper.add_effect(xpow[p], dim, np.cos)
#         xpow_ = helper.add_effect(xpow_, dim, np.sin)
#         xpow_ = helper.add_effect(xpow_, dim, lambda x: np.log(x + 1))
        w, accuracy, rmse_tr, rmse_te = train_model_ridge_cross(xpow_, ys[p], lambdas=lambdas)
        ws.append(w)
        accuracies.append(accuracy)
        rmse_trs.append(rmse_tr)
        rmse_tes.append(rmse_te)
        
    return ws, accuracies, rmse_trs, rmse_tes

In [31]:
temp = train_all_models(x, y, parts=[0,1])

In [32]:
denom = sum(data[p][0].shape[0] for p in part)
num = sum(data[p][0].shape[0] * temp[1][p] for p in part)
print(num/denom)

0.810878677723


In [14]:
test_data = []
for p in part:
    test_data.append(helper.load_clean_data(f'../../data/clean/x_test_leo_{p}.npy', f'../../data/clean/ids_test_leo_{p}.npy'))

In [15]:
x_test = [test_data[p][0] for p in part]
id_test = [test_data[p][1] for p in part]


In [17]:
print(id_test[1].shape)
print(x_test[1].shape)

(340780,)
(340780, 30)


In [18]:
xpow_test = [helper.addConstant(helper.toDeg(x_test[p], 2)) for p in part]

In [19]:
weights = temp[0]
weights[0].shape


(35,)

In [20]:
y_test = []
for p in part:
    dim = xpow_test[p].shape[1]
    xpow_test_ = xpow_test[p]
#     xpow_test_ = helper.add_effect(xpow_test[p], dim, np.cos)
#     xpow_test_ = helper.add_effect(xpow_test_, dim, np.sin)
    y_test.append(helper.predict_labels(weights[p],xpow_test_))

In [21]:
y_ret = np.concatenate(y_test)

In [22]:
ids_ret = np.concatenate(id_test)

In [23]:
y_ret = helper.changeYfromBinary(y_ret)

In [24]:
helper.create_csv_submission(ids=ids_ret,y_pred=y_ret,name='../../data/subm_leo_4.csv')